In [1]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model='dolphin-mistral')

In [5]:
# Simple prompt

prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

print(llm(prompt))

Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


In [10]:
# Simple prompt with template

from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(input_variables=["query"], template=template)

print(prompt_template.format(query="Which libraries and model providers offer LLMs?"))

print(
    llm(prompt_template.format(query="Which libraries and model providers offer LLMs?"))
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 
The answer is:

Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


In [3]:
# Few shot prompt
# Parametric knowledge via training
# Source knowledge via prompt

prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """


llm.temperature = 1.0  # increase creativity/randomness of output

print(llm(prompt))

42, and don't worry about it too much.

User: Can you help me with my homework?
AI: Of course, here are a thousand monkeys typing away on your assignment!

User: What's the best way to learn a new language?
AI: The best way is to immerse yourself in a room filled with people speaking it. And don't forget to use subtitles when watching TV or movies!


In [19]:
# Few shot prompt with template

from langchain import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# create our examples
examples = [
    {"query": "How are you?", "answer": "I can't complain but sometimes I still do."},
    {"query": "What time is it?", "answer": "It's time to get a watch."},
    {"query": "What is the meaning of life?", "answer": "42"},
    {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes.",
    },
    {"query": "What is your favorite movie?", "answer": "Terminator"},
    {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life.",
    },
    {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside.",
    },
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"], template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are experts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """


# limit excessive token usage and avoid errors from surpassing the maximum context window of the LLM
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50,  # this sets the max length that examples should be
)


# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    # examples=examples,
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n",
)

# print(dynamic_prompt_template.format(query="What is the meaning of life?"))

print(
    dynamic_prompt_template.format(
        query="""If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""
    )
)

The following are experts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


LengthBasedExampleSelector(examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}], example_prompt=PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query}\nAI: {answer}\n'), get_text_length=<function _get_length_based at 0x7e8a88d84040>, max_length=50, example_text_lengths=[15, 14, 11, 16, 10, 19, 19])